In [159]:
import numpy as np
from google.colab import files
from termcolor import colored

syntax = 'python' # or 'matlab' or 'cpp'
lineEnd = '\\r\\n'
comment = '!'

In [181]:
# CHOOSE INP FILE
uploaded = files.upload()
text = str(uploaded[ list(uploaded.keys())[0] ])

Saving chemkin_AR_HE.inp to chemkin_AR_HE (4).inp


In [182]:
# FUNCTIONS
# Splits string by separator, and removes empty elements from the resulting list
def separate(string: str, separator: str=' ') -> list:
  ret = string.split(separator)
  ret.append('')
  while ret[-1] == '':
    ret.remove('')
  return ret

# Finds the first line containing lookingFor
def find(lookingFor: str, lines: list) -> int:
  i = 0
  while i < len(lines) and comment in lines[i]:
    i += 1
  while i < len(lines) and not lookingFor in lines[i]:
    i += 1
  if i >= len(lines):
    print(colored(f'Error in find(), \'{lookingFor}\' not found', 'red'))
  return i

# Changes order of lines in array, so that it will be ordered as new instead of original
def rearrange(array: list, original: list, new: list) -> list:
  newArray = []
  if len(array) != len(original) or len(array) != len(new):
    print(colored(f'Warning in rearrange(), lists have different lengths', 'yellow'))
  for orig in original:
    if not orig in new:
      print(colored(f'Warning in rearrange(), \'{orig}\' is in original, but not in new', 'yellow'))
  for i in range(len(new)):
    index = 0
    if new[i] in original:
      index = original.index(new[i])
    else:
      print(colored(f'Error in rearrange(), \'{new[i]}\' is in new, but not in original', 'red'))
    newArray.append(array[index])
  return newArray

# Prints a 1D or 2D array or list
# Can use differents syntaxes (python, matlab, cpp)
# Can leave comments above columns and after lines
def printArray(array, syntax='python', width=0, comments=[], columns=[]):
  if type(array) == np.ndarray:
    array = list(array)
  if type(array[0]) == np.ndarray:
    for i, x in enumerate(array):
      array[i] = list(x)

  separator = ','
  arrOpener = '['
  arrCloser = ']'
  lineOpener = '['
  lineCloser = '],'
  remark = '#'
  if syntax == 'matlab':
    separator = ','
    arrOpener = '['
    arrCloser = ']'
    lineOpener = ''
    lineCloser = ';'
    remark = '%'
  if syntax == 'cpp':
    separator = ','
    arrOpener = '{'
    arrCloser = '};'
    lineOpener = '{'
    lineCloser = '},'
    remark = '//'

  if isinstance(array[0], list):
    print(arrOpener)
    if columns != []:
      print(f'\t{remark}', end='')
      for col in columns:
        print(f'{col: >{width}}', end=' ')
      print('')
    for i, x in enumerate(array):
      print('\t' + lineOpener, end='')
      for y in x:
        if type(y) == str:
          y = '"' + y + '"'
        print(f'{y:>{width}}', end=separator)
      if i != len(array)-1:
        print('\b' + lineCloser, end='')
      else:
        print('\b' + lineCloser + '\b ', end='')
      if len(comments) == len(array):
        print(f'    {remark} {comments[i]}', end='')
      print('')
    print(arrCloser)
  else:
    print(arrOpener, end='')
    for i, x in enumerate(array):
      if type(x) == str:
        x = '"' + x + '"'
      if len(comments) == len(array):
        print(f'{x:>{width}}', end=separator)
        print(f'    {remark} {comments[i]}')
      else:
        print(f'{x:>{width}}', end=separator)
    if len(comments) == len(array):
      print('\b' + '\n' + arrCloser)
    else:
      print('\b' + arrCloser)

In [183]:
# ELEMENTS AND SPECIES
text = text.replace('\\t', ' ')
lines = separate(text, lineEnd)
lines.append('END')
for i in range(len(lines)):
  lines[i] = lines[i].upper()
# ELEMENTS
i = find('ELEMENTS', lines) + 1
elements = separate(lines[i], ' ')

# SPECIES
i = find('SPECIES', lines) + 1
species = []
while not 'END' in lines[i]:
  if lines[i][0] != comment:
    species += separate(lines[i], ' ')
  i += 1

components = np.zeros((len(species), len(elements)), dtype=np.int32)
digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for i, specie in enumerate(species):
  specie += '  '
  specie = specie.replace('EX', '')
  while len(specie) > 0 and specie[0] != ' ':
    if specie[0:2] in elements:
      if specie[2] in digits:
        components[i][elements.index(specie[0:2])] = int(specie[2])
        specie = specie[3:]
      else:
        components[i][elements.index(specie[0:2])] = 1
        specie = specie[2:]
    elif specie[0] in elements:
      if specie[1] in digits:
        components[i][elements.index(specie[0])] = int(specie[1])
        specie = specie[2:]
      else:
        components[i][elements.index(specie[0])] = 1
        specie = specie[1:]
    else:
      specie = specie[2:]

print(colored(f'elements({len(elements)}) = ', attrs=['bold']))
printArray(elements, syntax, 0)
print(colored(f'species({len(species)}) = ', attrs=['bold']))
printArray(species, syntax, 0)
print(colored(f'components = ', attrs=['bold']))
printArray(components, syntax, 2, species, elements)

elements(6) = 
["C","H","N","O","AR","HE",]
species(12) = 
["H","H2","O","O2","OH","H2O","N2","HO2","H2O2","AR","HE","OHEX",]
components = 
[
	# C  H  N  O AR HE 
	[ 0, 1, 0, 0, 0, 0,],    # H
	[ 0, 2, 0, 0, 0, 0,],    # H2
	[ 0, 0, 0, 1, 0, 0,],    # O
	[ 0, 0, 0, 2, 0, 0,],    # O2
	[ 0, 1, 0, 1, 0, 0,],    # OH
	[ 0, 2, 0, 1, 0, 0,],    # H2O
	[ 0, 0, 2, 0, 0, 0,],    # N2
	[ 0, 1, 0, 2, 0, 0,],    # HO2
	[ 0, 2, 0, 2, 0, 0,],    # H2O2
	[ 0, 0, 0, 0, 1, 0,],    # AR
	[ 0, 0, 0, 0, 0, 1,],    # HE
	[ 0, 1, 0, 1, 0, 0,],     # OHEX
]


In [184]:
# THERMO ALL
TempRange = [] 
a_low = [] 
a_high = [] 
a = [] 
materials = [] 

i = find('THERMO ALL', lines) + 2

while lines[i][0] != comment and lines[i] != 'END':
  firstLine = separate(lines[i], ' ')
  if comment in firstLine: 
    firstLine.remove(comment) 
  if not firstLine[0] in materials:
    materials.append(firstLine[0])
    if firstLine[-2] == '0':
      firstLine = firstLine[:-2]
    else:
      firstLine = firstLine[:-1]
    TempRange.append( [float(firstLine[-3]), float(firstLine[-2]), float(firstLine[-1])] )

    otherLines = [] 
    for j in range(1, 4): 
      line = lines[i+j] 
      line = line[:-2] 
      line = line.replace('E-', '_') 
      line = line.replace('-', ' -') 
      line = line.replace('_', 'E-')
      otherLines += separate(line, ' ')

    arrayLine = []
    for num in otherLines: 
      arrayLine += [float(num)]
    a.append(arrayLine[0:14]) 
    a_high.append(arrayLine[0:7]) 
    a_low.append(arrayLine[7:14])

  i += 4

TempRange = rearrange(TempRange, materials, species)
a = rearrange(a, materials, species)
a_low = rearrange(a_low, materials, species)
a_high = rearrange(a_high, materials, species)

print(colored(f'TempRange = ', attrs=['bold']))
printArray(TempRange, syntax, 8, species, ['T_low', 'T_high', 'T_mid'])
print(colored(f'a_low = ', attrs=['bold']))
printArray(a_low, syntax, 16, species, ['a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'a_6', 'a_7'])
print(colored(f'a_high = ', attrs=['bold']))
printArray(a_high, syntax, 16, species, ['a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'a_6', 'a_7'])

TempRange = 
[
	#   T_low   T_high    T_mid 
	[   200.0,  6000.0,  1000.0,],    # H
	[   200.0,  6000.0,  1000.0,],    # H2
	[   200.0,  6000.0,  1000.0,],    # O
	[   200.0,  6000.0,  1000.0,],    # O2
	[   200.0,  6000.0,  1000.0,],    # OH
	[   200.0,  6000.0,  1000.0,],    # H2O
	[   200.0,  6000.0,  1000.0,],    # N2
	[   200.0,  5000.0,  1000.0,],    # HO2
	[   200.0,  6000.0,  1000.0,],    # H2O2
	[   200.0,  6000.0,  1000.0,],    # AR
	[   200.0,  6000.0,  1000.0,],    # HE
	[   300.0,  5000.0,  1000.0,],     # OHEX
]
a_low = 
[
	#             a_1              a_2              a_3              a_4              a_5              a_6              a_7 
	[             2.5,             0.0,             0.0,             0.0,             0.0,        25473.66,     -0.44668285,],    # H
	[      2.34433112,   0.00798052075,  -1.9478151e-05,  2.01572094e-08, -7.37611761e-12,     -917.935173,     0.683010238,],    # H2
	[       3.1682671,  -0.00327931884,  6.64306396e-06, -6.

In [185]:
# REACTIONS
reactions = []
numbers = []
A = []
b = []
E = []
ThirdBodyIndexes = []
alfa = []
PressureDependentIndexes = []
Troe = []
ReactConts = []

i = find('REACTIONS', lines) + 1

while not 'END' in lines[i]:
  if lines[i][0] != comment:
    reactionLine = lines[i]
    reactionLine = reactionLine.replace('>', '')
    reactionLine = reactionLine.replace('<', '') 
    reactionLine = reactionLine.replace(' =', '=')
    reactionLine = reactionLine.replace('= ', '=')
    reactionLine = reactionLine.replace('+ ', '+') 
    reactionLine = reactionLine.replace(' +', '+') 
    reactionLine = separate(reactionLine, ' ')
    if comment in reactionLine: 
      reactionLine.remove(comment)
    reactions.append(reactionLine[0])
    numbers.append(len(reactions))

    A.append(float(reactionLine[-3]))
    b.append(float(reactionLine[-2]))
    E.append(float(reactionLine[-1]))
    i += 1

    if 'DUP' in lines[i]: i+= 1
    if 'LOW' in lines[i] and 'TROE' in lines[i+1]:
      PressureDependentIndexes.append(numbers[-1]-1)
      line = lines[i]
      line = line.replace('LOW', '')
      line = line.replace('/', '')
      line = line.replace(comment, '')
      line = separate(line, ' ')
      ReactConts.append([float(line[-3]), float(line[-2]), float(line[-1])])
      i+= 1
    if 'TROE' in lines[i] and 'LOW' in lines[i-1]:
      line = lines[i]
      line = line.replace('TROE', '')
      line = line.replace('/', '')
      line = line.replace(comment, '')
      line = separate(line, ' ')
      if len(line) >= 4:
        Troe.append([float(line[-4]), float(line[-3]), float(line[-2]), float(line[-1])])
      else:
        Troe.append([float(line[-3]), 1e-30, float(line[-2]), float(line[-1])])
        print(colored(f'Warning, no T*** in line {i} (\'{lines[i]}\')', 'yellow'))
      i+= 1
    if 'LOW' in lines[i]: i+= 1
    if 'TROE' in lines[i]: i+= 1
    if 'PLOG' in lines[i]: i+= 1
    if 'PLOG' in lines[i]: i+= 1
    if 'PLOG' in lines[i]: i+= 1
    if '/' in lines[i]:
      ThirdBodyIndexes.append(numbers[-1]-1)
      line = lines[i]
      line = line.replace('/ ', ' ')
      line = line.replace('/', ' ')
      line = line.replace(comment, '')
      line = separate(line, ' ')
      alfa_line = np.ones((len(species)), dtype=np.float64)
      j = 0
      while j < len(line):
        if line[j] in species:
          alfa_line[species.index(line[j])] = float(line[j+1])
        else:
          print(colored(f'Warning, \'{line[j]}\' is not in species', 'yellow'))
        j += 2
      alfa.append(alfa_line)
      i+= 1
  else:
    i += 1

Warning, 'CO' is not in species
Warning, 'CO2' is not in species
Warning, 'CO' is not in species
Warning, 'CO2' is not in species
Warning, 'CO' is not in species
Warning, 'CO2' is not in species
Warning, 'CO' is not in species
Warning, 'CO2' is not in species
Warning, 'CO' is not in species
Warning, 'CO2' is not in species
Warning, 'CO2' is not in species
Warning, 'CO' is not in species


In [186]:
print(f'Number of reactions: {len(reactions)}')
print(colored(f'A = ', attrs=['bold']))
printArray(A, syntax, 0)
print(colored(f'b = ', attrs=['bold']))
printArray(b, syntax, 0)
print(colored(f'E = ', attrs=['bold']))
printArray(E, syntax, 0)
print(colored(f'ThirdBodyIndexes = ', attrs=['bold']))
printArray(ThirdBodyIndexes, syntax, 4)
print(colored(f'PressureDependentIndexes = ', attrs=['bold']))
printArray(PressureDependentIndexes, syntax, 4)
print(colored(f'ReactConts = ', attrs=['bold']))
printArray(ReactConts, syntax, 18, [f'{x:>2}. {reactions[x]}' for x in PressureDependentIndexes], ['A_0', 'b_0', 'E_0'])
print(colored(f'Troe = ', attrs=['bold']))
printArray(Troe, syntax, 18, [f'{x:>2}. {reactions[x]}' for x in PressureDependentIndexes], ['alfa',  'T***',  'T*',  'T**'])
print(colored(f'alfa = ', attrs=['bold']))
printArray(alfa, syntax, 8, [f'{x:>2}. {reactions[x]}' for x in ThirdBodyIndexes], species)

Number of reactions: 32
A = 
[5071200000000000.0,1255400.0,13193000.0,84999.0,4.9806e+18,6165000000000000.0,4.714e+18,1.4818e+24,4650000000000.0,2123100.0,57734000000000.0,32500000000000.0,958400000000.0,130000000000.0,1604800000000000.0,214800.0,24100000000000.0,9.7543e+19,9550000.0,1740000000000.0,75900000000000.0,15000000000000.0,5930000000000.0,2950000000000.0,108000000000.0,6010000000000.0,1310000000000.0,1690000000000.0,1450000.0,2100000000000.0,2750000000000.0,3230000000000.0,]
b = 
[-0.48596,2.27039,1.87803,2.26419,-1.21273,-0.5,-1.0,-2.53792,0.44,2.1133,0.0,0.0,0.42008,0.0,0.0,2.3219,0.0,-1.92495,2.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.5,0.5,0.5,]
E = 
[8116.1,3501.2,1585.8,-898.23,308.02,0.0,0.0,60.788,0.0,-817.68,86.07,0.0,-477.4,-820.33,7825.6,-1712.31,1997.99,4743.4,1997.99,160.04,3658.28,3007.05,-432.81,-223.45,-625.06,-384.5,-84.05,2081.03,0.0,-240.56,-487.17,-396.07,]
ThirdBodyIndexes = 
[   4,   5,   6,   7,   8,  15,  21,]
PressureDependentIndexes = 
[   8, 

In [187]:
# NU
nu_forward = np.zeros((len(reactions), len(species)), dtype=int)
nu_backward = np.zeros((len(reactions), len(species)), dtype=int)
digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for x, reaction in enumerate(reactions):
  reaction = reaction.replace('+M', '')
  reaction = reaction.replace('(+M)', '')
  reaction = reaction.replace('()', '')
  forward = separate(reaction, '=')[0]
  backward = separate(reaction, '=')[1]
  forward = separate(forward, '+')
  backward = separate(backward, '+')
  for f in forward:
    num = 1
    if f[0] in digits:
      num = int(f[0])
      f = f[1:]
    if f in species:
      nu_forward[x][species.index(f)] = num
    else:
      print(colored(f'Warning, \'{f}\' in reaction {x} (\'{reactions[x]}\') is not in species', 'yellow'))
    
  for b in backward:
    num = 1
    if b[0] in digits:
      num = int(b[0])
      b = b[1:]
    if b in species:
      nu_backward[x][species.index(b)] = num
    else:
      print(colored(f'Warning, \'{b}\' in reaction {x} (\'{reactions[x]}\') is not in species', 'yellow'))

print(colored(f'nu_forward = ', attrs=['bold']))
printArray(nu_forward, syntax, 4, [f'{x:>2}. {reaction}' for x, reaction in enumerate(reactions)], species)
print(colored(f'nu_backward = ', attrs=['bold']))
printArray(nu_backward, syntax, 4, [f'{x:>2}. {reaction}' for x, reaction in enumerate(reactions)], species)

Warning, 'HV' in reaction 28 ('OHEX=OH+HV') is not in species
Warning, 'CO2' in reaction 30 ('OHEX+CO2=OH+CO2') is not in species
Warning, 'CO2' in reaction 30 ('OHEX+CO2=OH+CO2') is not in species
Warning, 'CO' in reaction 31 ('OHEX+CO=OH+CO') is not in species
Warning, 'CO' in reaction 31 ('OHEX+CO=OH+CO') is not in species
nu_forward = 
[
	#   H   H2    O   O2   OH  H2O   N2  HO2 H2O2   AR   HE OHEX 
	[   1,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,],    #  0. H+O2=O+OH
	[   0,   1,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,],    #  1. O+H2=H+OH
	[   0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,],    #  2. OH+H2=H+H2O
	[   0,   0,   0,   0,   2,   0,   0,   0,   0,   0,   0,   0,],    #  3. 2OH=O+H2O
	[   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,],    #  4. 2H+M=H2+M
	[   0,   0,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,],    #  5. 2O+M=O2+M
	[   1,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,],    #  6. O+H+M=